## Force-based human's intent recognition: Raw-data-based classification

In this approach, using directly the data obtained from the sensor, the classification is done by means of a k-Nearest Neighbors (kNN) classifier with Dynamic Time Warping (DTW)[1] as metric. Particularly, we have used $k = 1$. 

Dynamic Time Warping is a time-dependent algorithm used to measure similarity between two temporal sequences which may vary in speed. For instance, similarities in polishing could be detected using DTW, even if the operator polishes faster or slower than in other occasions. DTW is a computational intense technique (quadratic time and memory complexity), however, there are some ways to accelerate its computation. In our case, we use the library Fast DTW[2].
DTW is meant to be utilized for uni-variate time series, which is not our case, since we have six sensor's axes. From the literature, we know at least two obvious approaches to tackle this and generalize DTW for multi-dimensional time series: *dependent* and *independent* DTW[3]. 
kNN classifier is taken from \textit{scikit learn} library. Since default implementations of both, kNN and Fast DTW, do not allow to work with multi-dimensional time series, it was necessary to adapt the used libraries. Apart from those modifications, we used the values set by default.

#### Note
In this notebook, you can train your classifier with cross validation without replacement and evaluate the results. Please, if you want to do the training with a pre-defined dataset and generate a final model, go to the another notebook of this folder. 

In [ ]:
# headers
import time
import copy
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt

from scripts.utils_general import *
from scripts.utils_evaluation import *
from scripts.utils_data_process import *
from scripts.utils_evaluation_global_variables import *

# headers dtw
from fastdtw._fastdtw import fastdtw 
from scipy.spatial.distance import euclidean
from scripts.kNN_wrapper import KNeighborsClassifierWrapper
from sklearn.metrics import accuracy_score, f1_score, recall_score, mean_squared_error, precision_score

In [ ]:
# variables for processing the data
type_of_dataset = 'natural' # natural or mechanical
labels = {0:'grab', 1:'move', 2:'polish'}
dataset_folder = '../data/'

training_portion = 0.75


# parameters for data length
number_of_measurements = 350 # size of the window
final_signal_size_percentage = 0.02 # percentage of the total signal which is kept after subsampling
step = int(round(1/final_signal_size_percentage)) # for subsampling


# multi-dimensional DTW
dtw_d = lambda a, b: sum((a - b) ** 2) ** 0.5
dtw_i = lambda a, b: sum(((a - b) ** 2) ** 0.5)

In [ ]:
# used metrics
def fastdtw_multivariate(x_, y_, dist_):
    distance = fastdtw(x_, y_, dist=dist_)[0]
    return distance

# dtw metrics
dtw_d_metric_ = lambda a, b: fastdtw_multivariate(a, b, dist_=dtw_d)
dtw_i_metric_ = lambda a, b: fastdtw_multivariate(a, b, dist_=dtw_i)

In [ ]:
processed_data = read_dataset_(dataset_folder, type_of_dataset, labels)

## Evaluation

In [ ]:
loop_evaluations = 1

for i in range (0, loop_evaluations):
    
    
    data_training, data_test = pick_training_dataset_randomly_(processed_data, training_portion, \
                                                               number_of_measurements, step, normalize=False)
    #"""
    # dependent DTW
    print "------ DTW d", i
    neighbors = 9
    metric_name_to_use_ = 'fastDTW'

    knn_dependent_dtw_ = KNeighborsClassifierWrapper(n_neighbors=neighbors, metric=dtw_d_metric_, algorithm='brute')

    knn_dependent_dtw_.fit(data_training['dtw_data'], data_training['training_labels'])
    print "fit done.."
    
    start_time = time.time()
    
    predictions_d = knn_dependent_dtw_.predict(data_test['dtw_data'])
    dtw_d_results['total_time'].append(time.time() - start_time)
    print "predict done.."
    
    dtw_d_results = evaluate_classification_performance_(data_test['test_labels'], predictions_d, dtw_d_results)

    print "evaluation done.."
    start_time = time.time()
    prediction = knn_dependent_dtw_.predict(data_test['dtw_data'][:1])
    dtw_d_results['time_one_sample'].append(time.time() - start_time)
        
    
    # independent DTW
    print "------ DTW i"
    knn_independent_dtw_ = KNeighborsClassifierWrapper(n_neighbors=neighbors, metric=dtw_i_metric_, algorithm='brute')

    knn_independent_dtw_.fit(data_training['dtw_data'], data_training['training_labels'])
    print "fit done.."
    
    start_time = time.time()
    predictions_i = knn_independent_dtw_.predict(data_test['dtw_data'])
    dtw_i_results['total_time'].append(time.time() - start_time)
    print "predict done.."
    
    dtw_i_results = evaluate_classification_performance_(data_test['test_labels'], predictions_i, dtw_i_results)
    
    print "evaluation done.."
    start_time = time.time()
    prediction = knn_independent_dtw_.predict(data_test['dtw_data'][:1])
    dtw_i_results['time_one_sample'].append(time.time() - start_time)
    
    
    #"""

In [ ]:
dtw_d_results_mean = dict()
dtw_d_results_std = dict()
dtw_i_results_mean = dict()
dtw_i_results_std = dict()

for key, value in dtw_d_results.items():
    if key == 'confusion_matrix':
        dtw_d_results_mean[key] = np.mean(value, axis=0)
        dtw_d_results_std[key] = np.std(value, axis=0)
    else: 
        dtw_d_results_mean[key] = np.mean(value)
        dtw_d_results_std[key] = np.std(value)

for key, value in dtw_i_results.items():
    if key == 'confusion_matrix':
        dtw_i_results_mean[key] = np.mean(value, axis=0)
        dtw_i_results_std[key] = np.std(value, axis=0)
    else: 
        dtw_i_results_mean[key] = np.mean(value)
        dtw_i_results_std[key] = np.std(value)

In [ ]:
all_results = dict()


all_results['dtw_d_mean'] = dtw_d_results_mean
all_results['dtw_d_std'] = dtw_d_results_std
all_results['dtw_i_mean'] = dtw_i_results_mean
all_results['dtw_i_std'] = dtw_i_results_std

In [ ]:
df_all_results = pd.DataFrame(all_results)
df_all_results.style

### References

[1] Berndt, D. J., & Clifford, J. (1994, July). Using dynamic time warping to find patterns in time series. In KDD workshop (Vol. 10, No. 16, pp. 359-370).
Bagnall, A., Bostrom, A., Large, J., & Lines, J. (2016). The great time series classification bake off: An experimental evaluation of recently proposed algorithms. extended version. arXiv preprint arXiv:1602.01711.

[2] Salvador, S., & Chan, P. (2007). Toward accurate dynamic time warping in linear time and space. Intelligent Data Analysis, 11(5), 561-580.

[3] Shokoohi-Yekta, M., Hu, B., Jin, H., Wang, J., & Keogh, E. (2017). Generalizing DTW to the multi-dimensional case requires an adaptive approach. Data mining and knowledge discovery, 31(1), 1-31.